In [6]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY") 

In [7]:
llm = ChatGroq(
    temperature = 0,
    groq_api_key = GROQ_API_KEY,
    model_name = "llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/lead-site-reliability-engineer-itc/job/R-62782")
page_data = loader.load().pop().page_content
print(page_data)






















Lead Site Reliability Engineer, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previou

In [9]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing following keys: `role`, `experience`, `skills`, `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input = {"page_data": page_data})
type(res.content)

str

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

{'role': 'Lead Site Reliability Engineer, ITC', 'experience': '7+ years of professional experience in software development, operations, or support', 'skills': ['Java', 'JavaScript', 'Kubernetes', 'Scala', 'Python', 'Golang', 'DNS', 'Networking', 'Virtualization', 'Linux', 'Docker', 'Serverless patterns', 'No-SQL database', 'RESTful APIs', 'ServiceNow', 'Jira', 'Jenkins', 'Splunk', 'New Relic', 'SignalFx'], 'description': 'The Lead Site Reliability Engineer will work with all engineering teams to ensure maximum availability, observability, reliability, security, and performance for Nike Digital Experiences. The candidate will belong to a horizontal domain called TechOps: Resilience Engineering and will interact with Engineering leaders, engineers, product teams, Scrum/Agile leads, Production support, business, and delivery teams.'}


In [11]:
type(json_res)

dict

In [12]:
import pandas as pd

df = pd.read_csv("portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [13]:
# Execute this cell if use the second version of code
# %pip install sentence-transformers

In [17]:
# import uuid
# import chromadb

# client = chromadb.PersistentClient('vectorstore')
# collection = client.get_or_create_collection(name="portfolio")

# if not collection.count():
#     for _, row in df.iterrows():
#         collection.add(documents=row["Techstack"],
#                        metadatas={"links": row["Links"]},
#                        ids=[str(uuid.uuid4())])

In [18]:

import uuid
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
client = chromadb.Client()

collection = client.get_or_create_collection(name="portfolio", embedding_function=embedding_function)

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas=[{"links": row["Links"]}],
            ids=[str(uuid.uuid4())]
        )

e:\Projects\ColdMailGenerator\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


In [19]:
links = collection.query(query_texts=["Experience in python", "Experience in React"], n_results = 2).get('metadatas', [])
links

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [20]:
job = json_res
job['skills']

['Java',
 'JavaScript',
 'Kubernetes',
 'Scala',
 'Python',
 'Golang',
 'DNS',
 'Networking',
 'Virtualization',
 'Linux',
 'Docker',
 'Serverless patterns',
 'No-SQL database',
 'RESTful APIs',
 'ServiceNow',
 'Jira',
 'Jenkins',
 'Splunk',
 'New Relic',
 'SignalFx']

In [21]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/v

In [22]:
job

{'role': 'Lead Site Reliability Engineer, ITC',
 'experience': '7+ years of professional experience in software development, operations, or support',
 'skills': ['Java',
  'JavaScript',
  'Kubernetes',
  'Scala',
  'Python',
  'Golang',
  'DNS',
  'Networking',
  'Virtualization',
  'Linux',
  'Docker',
  'Serverless patterns',
  'No-SQL database',
  'RESTful APIs',
  'ServiceNow',
  'Jira',
  'Jenkins',
  'Splunk',
  'New Relic',
  'SignalFx'],
 'description': 'The Lead Site Reliability Engineer will work with all engineering teams to ensure maximum availability, observability, reliability, security, and performance for Nike Digital Experiences. The candidate will belong to a horizontal domain called TechOps: Resilience Engineering and will interact with Engineering leaders, engineers, product teams, Scrum/Agile leads, Production support, business, and delivery teams.'}

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Shiva Kumar Souta, a business development executive at CodeVoid. CodeVoid is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of CodeVoid
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase CodeVoid's portfolio: {link_list}
        Remember you are Shiva Kumar Souta, BDE at CodeVoid.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Site Reliability Engineering Solutions for Nike Digital Experiences

Dear Hiring Manager,

I came across the job description for a Lead Site Reliability Engineer at Nike, and I'm excited to introduce CodeVoid, a renowned AI & Software Consulting company, as a potential partner to fulfill your requirements. With our extensive experience in software development, operations, and support, we're confident in our ability to ensure maximum availability, observability, reliability, security, and performance for Nike Digital Experiences.

Our team at CodeVoid possesses expertise in a wide range of technologies, including Java, JavaScript, Kubernetes, Scala, Python, Golang, DNS, Networking, Virtualization, Linux, Docker, Serverless patterns, No-SQL database, RESTful APIs, ServiceNow, Jira, Jenkins, Splunk, New Relic, and SignalFx. We've successfully empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened 